In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
import spacy
import pandas as pd
from transformer import Transformer 
import torch.optim as optim
import os
from tqdm import tqdm 
import torch
import numpy as np
import json
import string

In [2]:
data=pd.read_csv("Final_data.csv")

In [ ]:
# data =pd.read_csv("en-fr.csv")
# data = data.iloc[1:].reset_index(drop=True)
# #           Integer location based index
# # data.to_csv('Data.csv', index=False)
# data =pd.read_csv("Data.csv")

In [ ]:
# data.head()

In [ ]:
# data.shape

In [ ]:
# nulls_value = data.isnull().sum()
# # Create a dataframe of same shape as data is there is any none value then it is False
# # nulls_per_column -> Series
# print(nulls_value)

In [ ]:
# data['English'] = data['English'].str.replace(f"[{string.punctuation}]", "", regex=True).str.lower()
# data['French'] = data['French'].str.replace(f"[{string.punctuation}]", "", regex=True).str.lower()

In [ ]:
# data

In [ ]:
# def has_number_or_special(value):
#     return any(char.isdigit() for char in value) or any(char in string.punctuation for char in value)

# data = data[~data.apply(lambda col: col.map(has_number_or_special)).any(axis=1)]

# # Save the cleaned dataset
# #df_cleaned.to_csv('Final_data.csv', index=False)

In [ ]:
# data.shape

In [ ]:
# data1 = data.dropna()

In [ ]:
# nulls_value = data1.isnull().sum()
# # Create a dataframe of same shape as data is there is any none value then it is False
# # nulls_per_column -> Series
# print(nulls_value)

In [ ]:
# data1.to_csv('Final_Cleaned_Data.csv', index=False)

In [3]:
nlp_en = spacy.load('en_core_web_sm')
nlp_fr = spacy.load('fr_core_news_sm')

In [4]:
source_sentences = data.iloc[:, 0].tolist()
target_sentences = data.iloc[:, 1].tolist()

In [5]:
len(source_sentences)

228439

In [6]:
len(target_sentences)

228439

In [18]:
# cleaned_lst = [x for x in source_sentences if '\u202f' not in x and '\xa0' not in x and "\u00e0" not in x and "\u2009" not in x and "\u00e7" and '\xa0' not in x]

In [19]:
# cleaned_lst

In [20]:
# source_sentences =cleaned_lst

In [21]:
# cleaned_lst = [x for x in target_sentences if '\u202f' not in x and '\xa0' not in x and "\u00e0" not in x and "\u2009" not in x and "\u00e7" and '\xa0' not in x]

In [22]:
# target_sentences=cleaned_lst

In [23]:
# len(target_sentences)

In [24]:
# len(source_sentences)

In [25]:
# source_sentences =source_sentences[:184416]

In [26]:
# len(source_sentences)

In [27]:
# PERCENTILE = 99.8
# print( f"{PERCENTILE}th percentile length English: {np.percentile([len(x) for x in source_sentences], PERCENTILE)}" )
# print( f"{PERCENTILE}th percentile length French: {np.percentile([len(x) for x in target_sentences], PERCENTILE)}" )
     

In [7]:
# Pre Process with punctuatuion and spaces
def preprocess_text_spacy(text, nlp_model):

    doc = nlp_model(text)
    tokens = []
    for token in doc:
        if not token.is_punct and not token.is_space:
            tokens.append(token.text.lower())
    return tokens

# Define the build_vocab function to use preprocessed tokens
def build_vocab(sentences, max_vocab_size=50000):
    vocab = {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3}
    
    index = len(vocab)
    added_tokens = set(vocab.keys())
    
    for sentence in sentences:
        for token in sentence:
            if token not in added_tokens:
                print (index)
                if index >= max_vocab_size:
                    print(f"Max vocab size {max_vocab_size} reached.")
                    return vocab
                vocab[token] = index
                added_tokens.add(token)
                index += 1
                    
    return vocab

# Padding Tokens
def pad_sequence(tokens, max_len, pad_token='<PAD>'):
    # Take only till first max_len part of the sentence if sentence length is more or take the max_len
    padded_tokens = tokens[:max_len]
    padded_tokens += [pad_token] * (max_len - len(padded_tokens))
    return padded_tokens

# Tokens to IDs
def tokens_to_ids(tokens, vocab):
    return [vocab.get(token, vocab['<UNK>']) for token in tokens]

In [8]:
# Preprocess 
preprocessed_source = [preprocess_text_spacy(sentence, nlp_en) for sentence in source_sentences]
preprocessed_target = [preprocess_text_spacy(sentence, nlp_fr) for sentence in target_sentences]

KeyboardInterrupt: 

In [ ]:
# #Vocab 
# src_vocab = build_vocab(preprocessed_source)
# tgt_vocab = build_vocab(preprocessed_target)

In [ ]:
# def save_vocab(vocab, filepath):
#     # Open the file path in write mode in present or not present then create the file path.
#     #with ->It closes file is properly closed.
#     with open(filepath, 'w') as file:
#         json.dump(vocab, file, indent=4)

# save_vocab(src_vocab, 'src_vocab.json')
# save_vocab(tgt_vocab, 'tgt_vocab.json')

In [ ]:
with open('src_vocab.json', 'r') as src_file:
    src_vocab = json.load(src_file)

with open('tgt_vocab.json', 'r') as tgt_file:
    tgt_vocab = json.load(tgt_file)

In [ ]:
# Padding
max_seq_length =100
padded_source = [pad_sequence(sentence, max_seq_length) for sentence in preprocessed_source]
padded_target = [pad_sequence(sentence, max_seq_length) for sentence in preprocessed_target]

# Tokenization
source_token_ids = [tokens_to_ids(sentence, src_vocab) for sentence in padded_source]
target_token_ids = [tokens_to_ids(sentence, tgt_vocab) for sentence in padded_target]

In [ ]:
# src_vocab

In [ ]:
# tgt_vocab

In [ ]:
class TranslationDataset(Dataset):
    def __init__(self, source_ids, target_ids):
        self.source_ids = source_ids
        self.target_ids = target_ids

    def __len__(self):
        return len(self.source_ids)

    def __getitem__(self, idx):
        #              Torch.long is used for 64-bit integer values. 
        source = torch.tensor(self.source_ids[idx], dtype=torch.long)
        target = torch.tensor(self.target_ids[idx], dtype=torch.long)
        return source, target

In [ ]:
full_dataset = TranslationDataset(source_token_ids, target_token_ids)

In [ ]:
len(full_dataset)

228439

In [ ]:
train_loader = DataLoader(full_dataset, batch_size=50, shuffle=True)

In [ ]:
for batch_idx, (source_batch, target_batch) in enumerate(train_loader):
    print(f"Batch {batch_idx} - Source Batch: {source_batch.size()}, Target Batch: {target_batch.size()}")

Batch 0 - Source Batch: torch.Size([50, 100]), Target Batch: torch.Size([50, 100])
Batch 1 - Source Batch: torch.Size([50, 100]), Target Batch: torch.Size([50, 100])
Batch 2 - Source Batch: torch.Size([50, 100]), Target Batch: torch.Size([50, 100])
Batch 3 - Source Batch: torch.Size([50, 100]), Target Batch: torch.Size([50, 100])
Batch 4 - Source Batch: torch.Size([50, 100]), Target Batch: torch.Size([50, 100])
Batch 5 - Source Batch: torch.Size([50, 100]), Target Batch: torch.Size([50, 100])
Batch 6 - Source Batch: torch.Size([50, 100]), Target Batch: torch.Size([50, 100])
Batch 7 - Source Batch: torch.Size([50, 100]), Target Batch: torch.Size([50, 100])
Batch 8 - Source Batch: torch.Size([50, 100]), Target Batch: torch.Size([50, 100])
Batch 9 - Source Batch: torch.Size([50, 100]), Target Batch: torch.Size([50, 100])
Batch 10 - Source Batch: torch.Size([50, 100]), Target Batch: torch.Size([50, 100])
Batch 11 - Source Batch: torch.Size([50, 100]), Target Batch: torch.Size([50, 100])
Ba

In [ ]:
src_vocab_size = len(src_vocab)
tgt_vocab_size = len(tgt_vocab)
d_model = 512
heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100 
dropout = 0.1
batch_size = 50
num_epochs = 20

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Transformer(src_vocab_size, tgt_vocab_size, d_model, heads, num_layers, d_ff, max_seq_length, dropout).to(device)

In [9]:
model

Transformer(
  (encoder_embedding): Embedding(16317, 512)
  (decoder_embedding): Embedding(29592, 512)
  (positional_encoding): PositionalEncoding()
  (encoder_layers): ModuleList(
    (0-5): 6 x EncoderLayer(
      (self_attn): MultiHeadAttention(
        (W_v): Linear(in_features=512, out_features=512, bias=True)
        (W_k): Linear(in_features=512, out_features=512, bias=True)
        (W_q): Linear(in_features=512, out_features=512, bias=True)
        (W_o): Linear(in_features=512, out_features=512, bias=True)
      )
      (feed_forward): PositionWiseFeedForward(
        (fcnn1): Linear(in_features=512, out_features=2048, bias=True)
        (fcnn2): Linear(in_features=2048, out_features=512, bias=True)
        (relu): ReLU()
      )
      (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (decoder_layers): ModuleList(
    (0-5): 6 x DecoderLa

In [10]:
#             ignore index = 0-> Handing Padding Tokens
criterion = torch.nn.CrossEntropyLoss(ignore_index=0)

optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [113]:
#Training
def train(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    
    # Progess bar
    loop = tqdm(dataloader, leave=True)
    
    for src, tgt in loop:
        src, tgt = src.to(device), tgt.to(device)
        
        #EOS
        tgt_input = tgt[:, :-1] 
        #SOS
        tgt_output = tgt[:, 1:] 

        optimizer.zero_grad()
        output = model(src, tgt_input)

        #Reshaping to the last dimension
        output = output.reshape(-1, output.size(-1))
        tgt_output = tgt_output.reshape(-1)
        loss = criterion(output, tgt_output)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        
        loop.set_description(f"Loss: {loss.item():.4f}")
    
    return total_loss / len(dataloader)

In [165]:
# Load the checkpoint model
checkpoint_dir = 'Checkpoint'
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_files = [f for f in os.listdir(checkpoint_dir) if f.endswith('.pth')]

latest_epoch = -1
latest_checkpoint_file = None

for file in checkpoint_files:
    try:
        epoch = int(file.split('_')[-1].split('.')[0])
        if epoch > latest_epoch:
            latest_epoch = epoch
            latest_checkpoint_file = file
    except ValueError:
        continue

print(f'Latest epoch found: {latest_epoch}')

start_epoch = latest_epoch

if latest_checkpoint_file:
    checkpoint_path = os.path.join(checkpoint_dir, latest_checkpoint_file)
    print(f'Loading checkpoint from {checkpoint_path}')
    
    checkpoint = torch.load(checkpoint_path, weights_only=True)

    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint.get('epoch', latest_epoch)  
    
    model.to(device)

Latest epoch found: 20
Loading checkpoint from Checkpoint\checkpoint_epoch_20.pth


In [145]:
# Training loop
for epoch in range(start_epoch, num_epochs):
    print(f'Epoch {epoch+1}')
    train_loss = train(model, train_loader, optimizer, criterion, device)
    print(f'Loss: {train_loss:.3f}')

    checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch+1}.pth")
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': train_loss,
    }, checkpoint_path)
    print(f'Saved checkpoint to {checkpoint_path}')
torch.save(model, 'Model.pth')

Epoch 19


Loss: 0.2648: 100%|██████████| 4569/4569 [40:32<00:00,  1.88it/s]


Loss: 0.303
Saved checkpoint to Checkpoint\checkpoint_epoch_19.pth
Epoch 20


Loss: 0.2991: 100%|██████████| 4569/4569 [40:33<00:00,  1.88it/s]


Loss: 0.303
Saved checkpoint to Checkpoint\checkpoint_epoch_20.pth


In [146]:
model = torch.load('Model.pth')

In [166]:
def predict(model, src_tensor, max_len, device, id_to_word):
    model.eval()
    tgt_tokens = [src_vocab['<SOS>']]  
    tgt_tensor = torch.tensor([tgt_tokens]).to(device)
 
    predictions = []
 
    with torch.no_grad():
        for _ in range(max_len):
            output = model(src_tensor, tgt_tensor)
            next_token = output.argmax(dim=-1)[:, -1].item()
            predictions.append(next_token)
           
            tgt_tokens.append(next_token)
            tgt_tensor = torch.tensor([tgt_tokens]).to(device)  
 
    predicted_words = [id_to_word.get(token_id, '<UNK>') for token_id in predictions]
    predicted_sentence = ' '.join(predicted_words)
    return predicted_sentence

In [167]:

def generate_predicted_sentence(model, src_sentence, src_vocab, tgt_vocab, preprocess, nlp, predict, device, max_len=None):
    num = src_sentence.count(' ')
    id_to_word = {id_: word for word, id_ in tgt_vocab.items()}
    src_tensor = torch.tensor(tokens_to_ids(preprocess(src_sentence, nlp), src_vocab)).unsqueeze(0).to(device)
    predicted_sentence = predict(model, src_tensor, max_len=100, device=device, id_to_word=id_to_word)
    a=0
    if (predicted_sentence.__contains__('<UNK>')):
        a=a+1
    predicted_sentence=predicted_sentence.replace('<UNK>','')
    
    def get_prediction(predicted_sentence, num, a):
        words = predicted_sentence.split()  
        return ' '.join(words[:num + a]) 
    ans=get_prediction(predicted_sentence, num, a)
    return ans

In [154]:
src_sentence = "go there"
predicted_sentence = generate_predicted_sentence(model=model,src_sentence=src_sentence,src_vocab=src_vocab,tgt_vocab=tgt_vocab,preprocess=preprocess_text_spacy,
                                                    nlp=nlp_en,predict=predict,device=device)
print("Input Sentence:" ,src_sentence)
print(f'Predicted translation: {predicted_sentence}')

Input Sentence: go there
Predicted translation: bas bas


In [168]:
src_sentence = "go there"
predicted_sentence = generate_predicted_sentence(model=model,src_sentence=src_sentence,src_vocab=src_vocab,tgt_vocab=tgt_vocab,preprocess=preprocess_text_spacy,
                                                    nlp=nlp_en,predict=predict,device=device)
print("Input Sentence:" ,src_sentence)
print(f'Predicted translation: {predicted_sentence}')

Input Sentence: go there
Predicted translation: bas bas


In [153]:
src_sentence = "i am here"
predicted_sentence = generate_predicted_sentence(model=model,src_sentence=src_sentence,src_vocab=src_vocab,tgt_vocab=tgt_vocab,preprocess=preprocess_text_spacy,
                                                    nlp=nlp_en,predict=predict,device=device)
print("Input Sentence:" ,src_sentence)
print(f'Predicted translation: {predicted_sentence}')

Input Sentence: i am here
Predicted translation: je suis ici


In [169]:
src_sentence = "i am here"
predicted_sentence = generate_predicted_sentence(model=model,src_sentence=src_sentence,src_vocab=src_vocab,tgt_vocab=tgt_vocab,preprocess=preprocess_text_spacy,
                                                    nlp=nlp_en,predict=predict,device=device)
print("Input Sentence:" ,src_sentence)
print(f'Predicted translation: {predicted_sentence}')

Input Sentence: i am here
Predicted translation: je suis ici


In [156]:
src_sentence = "here "
predicted_sentence = generate_predicted_sentence(model=model,src_sentence=src_sentence,src_vocab=src_vocab,tgt_vocab=tgt_vocab,preprocess=preprocess_text_spacy,
                                                    nlp=nlp_en,predict=predict,device=device)
print("Input Sentence:" ,src_sentence)
print(f'Predicted translation: {predicted_sentence}')

Input Sentence: here 
Predicted translation: ici


In [170]:
src_sentence = "here "
predicted_sentence = generate_predicted_sentence(model=model,src_sentence=src_sentence,src_vocab=src_vocab,tgt_vocab=tgt_vocab,preprocess=preprocess_text_spacy,
                                                    nlp=nlp_en,predict=predict,device=device)
print("Input Sentence:" ,src_sentence)
print(f'Predicted translation: {predicted_sentence}')

Input Sentence: here 
Predicted translation: ici


In [171]:
src_sentence = "I am a boy "
predicted_sentence = generate_predicted_sentence(model=model,src_sentence=src_sentence,src_vocab=src_vocab,tgt_vocab=tgt_vocab,preprocess=preprocess_text_spacy,
                                                    nlp=nlp_en,predict=predict,device=device)
print("Input Sentence:" ,src_sentence)
print(f'Predicted translation: {predicted_sentence}')

Input Sentence: I am a boy 
Predicted translation: un garçon que je suis


In [172]:
src_sentence = "I am a boy "
predicted_sentence = generate_predicted_sentence(model=model,src_sentence=src_sentence,src_vocab=src_vocab,tgt_vocab=tgt_vocab,preprocess=preprocess_text_spacy,
                                                    nlp=nlp_en,predict=predict,device=device)
print("Input Sentence:" ,src_sentence)
print(f'Predicted translation: {predicted_sentence}')

Input Sentence: I am a boy 
Predicted translation: un garçon que je suis


In [159]:
src_sentence = "Welcome to new world "
predicted_sentence = generate_predicted_sentence(model=model,src_sentence=src_sentence,src_vocab=src_vocab,tgt_vocab=tgt_vocab,preprocess=preprocess_text_spacy,
                                                    nlp=nlp_en,predict=predict,device=device)
print("Input Sentence:" ,src_sentence)
print(f'Predicted translation: {predicted_sentence}')

Input Sentence: Welcome to new world 
Predicted translation: rien nouveau monde est nouveau


In [173]:
src_sentence = "Welcome to new world "
predicted_sentence = generate_predicted_sentence(model=model,src_sentence=src_sentence,src_vocab=src_vocab,tgt_vocab=tgt_vocab,preprocess=preprocess_text_spacy,
                                                    nlp=nlp_en,predict=predict,device=device)
print("Input Sentence:" ,src_sentence)
print(f'Predicted translation: {predicted_sentence}')

Input Sentence: Welcome to new world 
Predicted translation: rien nouveau monde est nouveau


In [161]:
src_sentence = "Lives "
predicted_sentence = generate_predicted_sentence(model=model,src_sentence=src_sentence,src_vocab=src_vocab,tgt_vocab=tgt_vocab,preprocess=preprocess_text_spacy,
                                                    nlp=nlp_en,predict=predict,device=device)
print("Input Sentence:" ,src_sentence)
print(f'Predicted translation: {predicted_sentence}')

Input Sentence: Lives 
Predicted translation: les


In [174]:
src_sentence = "Lives "
predicted_sentence = generate_predicted_sentence(model=model,src_sentence=src_sentence,src_vocab=src_vocab,tgt_vocab=tgt_vocab,preprocess=preprocess_text_spacy,
                                                    nlp=nlp_en,predict=predict,device=device)
print("Input Sentence:" ,src_sentence)
print(f'Predicted translation: {predicted_sentence}')

Input Sentence: Lives 
Predicted translation: les


In [162]:
src_sentence = "I lives in India"
predicted_sentence = generate_predicted_sentence(model=model,src_sentence=src_sentence,src_vocab=src_vocab,tgt_vocab=tgt_vocab,preprocess=preprocess_text_spacy,
                                                    nlp=nlp_en,predict=predict,device=device)
print("Input Sentence:" ,src_sentence)
print(f'Predicted translation: {predicted_sentence}')

Input Sentence: I lives in India
Predicted translation: inde je suis inde


In [175]:
src_sentence = "I lives in India"
predicted_sentence = generate_predicted_sentence(model=model,src_sentence=src_sentence,src_vocab=src_vocab,tgt_vocab=tgt_vocab,preprocess=preprocess_text_spacy,
                                                    nlp=nlp_en,predict=predict,device=device)
print("Input Sentence:" ,src_sentence)
print(f'Predicted translation: {predicted_sentence}')

Input Sentence: I lives in India
Predicted translation: inde je suis inde


In [185]:
src_sentence = "My name is Rohit"
predicted_sentence = generate_predicted_sentence(model=model,src_sentence=src_sentence,src_vocab=src_vocab,tgt_vocab=tgt_vocab,preprocess=preprocess_text_spacy,
                                                    nlp=nlp_en,predict=predict,device=device)
print("Input Sentence:" ,src_sentence)
print(f'Predicted translation: {predicted_sentence}')

Input Sentence: My name is Rohit
Predicted translation: mon nom est bleu
